# Register a Model on Fiddler

This tutorial shows how to use XAI and Model Monitoring in Fiddler. 
The tutorial has three parts:
- upload your dataset
- register your model
- send prediction events

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.


In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Load Dataset

Here we will load in our baseline dataset from a csv called `p2p_loans.csv`. We will
also create a schema using this information.

In [ ]:
import pandas as pd
dataset_df = pd.read_csv(
    'https://raw.githubusercontent.com/fiddler-labs/fiddler-samples/master/content_root/samples/datasets/p2p_loans/p2p_loans.csv'
)
dataset_df

## Infer dataset schema

In [ ]:
# infer dataset schema
schema = fdl.DatasetInfo.from_dataframe(dataset_df, max_inferred_cardinality=1000)

# inspect schema and modify as needed
schema

## Upload Dataset

In [ ]:
dataset_id = 'loan_dataset'

# delete dataset if it is already available
client.delete_dataset(dataset_id)

# split dataset
train_df = dataset_df[:2000]
test_df = dataset_df[2001:]

# upload dataset
client.upload_dataset(
    dataset={
        'train': train_df,
        'test': test_df
    },
    dataset_id=dataset_id,
    info=schema,
)
dataset_info = client.get_dataset_info(dataset_id)
    
dataset_info


## Setup project

In [ ]:
project_id = 'tutorial'

if project_id not in client.list_projects():
    client.create_project(project_id)
    


## Create model schema


In [ ]:
model_id = 'loan_model'
target='loan_status'
features = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_range_low', 
            'total_acc', 'acc_open_past_24mths', 'addr_state']
decisions = ['home_ownership']
metadata = ['id', 'zip_code_prefix']

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    target=target,
    features=features,
    display_name='Loan Model',
    description='Loan Model registered using register_model API',
    decision_cols=decisions,
    metadata_cols=metadata,
    input_type=fdl.ModelInputType.TABULAR,
    model_task=fdl.ModelTask.BINARY_CLASSIFICATION,
    outputs=['probability_fully_paid'],
)

# inspect model info and modify as needed
model_info


## Register model

In [ ]:
# delete model if it is already available
client.delete_model(project_id, model_id, delete_prod=True)

# register model
client.register_model(project_id, model_id, dataset_id, model_info)


## Send Monitoring Events

In this step, we will be simulating traffic to send for our model monitoring by using 
[publish_event](https://docs.fiddler.ai/api-reference/python-package/#publish-event). 
This will be the equivalent of running our model separately on data, and either 
sending to Fiddler then, or saving this information to a log and sending at a later point.

For this demonstration, we will be going with a log-related approach. 
This log contains rows that have inputs and predictions. 
To most accurately simulate this as a time-series event, we will generate a timestamp and send an event every 5 minutes. Real data will ideally have a timestamp related to when the event took place; otherwise, the current 
time will be used.

We can send the inputs, outputs, targets as well as decisions variables.

**Note**: The timestamp must be in UTC milliseconds. See 
[here](https://docs.fiddler.ai/api-reference/python-package/#publish-event) for more details

In [ ]:
import datetime
import time
from IPython.display import clear_output
import random

result_df = pd.read_csv('https://raw.githubusercontent.com/fiddler-labs/fiddler-samples/master/content_root/samples/datasets/p2p_loans/p2p_production.log')

NUM_EVENTS_TO_SEND = 50

ONE_DAY_MS = 8.64e+7
event_time = round(time.time() * 1000) - (ONE_DAY_MS * 8)
interval = round((ONE_DAY_MS * 8) / NUM_EVENTS_TO_SEND)

# Convert this dataframe into a list of dictionary events, where each event is its own dictionary
event_list_dict = result_df.sample(n=NUM_EVENTS_TO_SEND).to_dict(orient='records') 

for ind, event_dict in enumerate(event_list_dict):
    event_time = event_time + random.randint(1, interval * 2)
    result = client.publish_event(project_id,
                                  model_id,
                                  event_dict,
                                  event_time_stamp=event_time,
                                  event_id=str(ind + 100),
                                  update_event=False)
    
    clear_output(wait = True)
    
    print(f'Sending {ind+1} / {NUM_EVENTS_TO_SEND} \n {event_dict}' )
    time.sleep(0.1)

**Note**: If we want to update the events later, we need to specify an `event_id`. To update an event, we need to call `publish_event` again with the same `event_id` and `update_event=True`.